In [166]:
import numpy as np
import pandas as pd
import datetime as dt
import requests
import io

In [68]:
data = pd.read_csv("Resources/cleaning-home.csv")

C:\Users\alana\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data = data.drop(columns=['Unnamed: 0'])
data['BABIP'] = (data.H - data.HR) / (data.AB - data.SO - data.HR + data.SF)
data.DaysOld = (pd.to_datetime(data.StartSeason) - pd.to_datetime(data.borndate))
data.DaysOld = round(data.DaysOld.dt.days, 0)
data['SB%'] = data['SB%'].fillna(0)

In [69]:
leveler = {'A+':3, 'A': 4, 'Rk':6, 'A-':5, 'AAA': 1, 'AA':2}
releveler = {3:'A+', 4:'A', 6:'Rk', 5:'A-', 1: 'AAA', 2 : 'AA'}

In [ ]:
url = "https://raw.githubusercontent.com/chadwickbureau/register/master/data/people.csv"
s=requests.get(url).content
table = pd.read_csv(io.StringIO(s.decode('utf-8')), dtype={'key_sr_nfl': object, 'key_sr_nba': object, 'key_sr_nhl': object})
#subset columns
cols_to_keep = ['name_last','name_first','key_mlbam', 'key_retro', 'key_bbref', 'key_fangraphs', 'mlb_played_first','mlb_played_last']
table = table[cols_to_keep]
#make these lowercase to avoid capitalization mistakes when searching
table['name_last'] = table['name_last'].str.lower()
table['name_first'] = table['name_first'].str.lower()
# Pandas cannot handle NaNs in integer columns. We need IDs to be ints for successful queries in statcast, etc. 
# Workaround: replace ID NaNs with -1, then convert columns to integers. User will have to understand that -1 is not a valid ID. 
table[['key_mlbam', 'key_fangraphs']] = table[['key_mlbam', 'key_fangraphs']].fillna(-1)
table[['key_mlbam', 'key_fangraphs']] = table[['key_mlbam', 'key_fangraphs']].astype(int) # originally returned as floats which is wrong

#filters out before 2005, which should cover the players we need
table = table[table.mlb_played_first > 2005]

In [ ]:
def playerid_lookup(last, first=None):
    # force input strings to lowercase
    last = last.lower()
    if first:
        first = first.lower()
    if first is None:
        results = table.loc[table['name_last']==last]
    else:
        results = table.loc[(table['name_last']==last) & (table['name_first']==first)]
    #results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']] = results[['key_mlbam', 'key_fangraphs', 'mlb_played_first', 'mlb_played_last']].astype(int) # originally returned as floats which is wrong
    results = results.reset_index().drop('index', 1)
    return results

In [70]:
data['Top'] = data.Level
data = data.replace({"Level" : leveler})

In [71]:
looker = data[['playerid', 'Level', 'Top']]
looker = looker.groupby('playerid').min()
looker = looker.reset_index()
looker = looker.drop(columns = ['Top'])
looker = looker.replace({"Level" : releveler})
looker.head()

In [72]:
final = data.merge(looker, on='playerid', how='left')
final = final.rename(columns={'Level_x': 'Level'})
final = final.rename(columns={'Level_y': 'TopLevel'})
final = final.drop(columns='Top')

In [73]:
look = final.groupby(['League', 'Level', 'year']).transform(lambda x: (x - x.mean()) / x.std())

In [225]:
last = final[['playerid', 'TopLevel', 'year', 'teamName', 'League', 'Level', 'orgName',
       'lastName', 'firstName', 'StartSeason', 'Bats', 'Throws', 'posit', 'borndate',
        'cityName', 'regionID', 'mlbid', 'PA']]

In [226]:
last[['AVG', 'SLG', 'OBP', 'OPS', 'Age', 'HT', 'WT', 'DEF', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP', 'DaysOld']] = look[['Bavg', 'Slg', 'obp', 'OPS', 'Age', 'HT', 'WT', 'Def', 'ISO', 'BB%', 'K%', 'SB%', 'BABIP', 'DaysOld']]

C:\Users\alana\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [228]:
last.loc[(last.mlbid > 0) & (last.year < 2010), 'TopLevel'] = "MLB"

C:\Users\alana\Anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [77]:
names = last[['firstName', 'lastName', 'year', 'playerid']]
names = names[names.year > 2009]
names['full'] = names.firstName + " " + names.lastName
names = names.drop_duplicates(subset='full')
names['id'] =  np.nan

In [83]:
id = []
for index, row in names.iterrows():
    try:
        test = playerid_lookup(row['lastName'], row['firstName'])
        id.append(test['key_fangraphs'][0])
    except IndexError:
        id.append(np.nan)

In [84]:
names['id'] = id

In [88]:
names = names[['playerid', 'id']]

In [90]:
last.to_csv("Resources/nocigar.csv")

In [216]:
names.id.loc[~names.id.isnull()] = 'MLB'  # not nan

In [230]:
last2 = last.merge(names, on='playerid', how = 'left')

In [232]:
last2.id.fillna(last2.TopLevel, inplace=True)

In [237]:
last2.to_csv("Resources/done.csv")

In [238]:
import pandas as pd

In [239]:
last2 = pd.read_csv("Resources/done.csv")

In [241]:
last2.TopLevel = last2.id
last2 = last2.drop(columns =['id', 'Unnamed: 0'])

In [243]:
last2.to_csv("Resources/done.csv")